# **Let's combine and find out some interesting things about our data** 

## Importing the necessary libraries

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px 
import warnings as warn 

sns.set_style("darkgrid")
warn.simplefilter("ignore")

In [3]:
orders = pd.read_csv("../db/Orders.csv").drop(columns=["Unnamed: 0"])
display(orders)

,OrderID,Date,Status,Fulfilment,SalesChannel,shipservicelevel,Style,SKU,Category,Size,...,Qty,currency,Amount,shipcity,shipstate,shippostalcode,shipcountry,promotionids,B2B,fulfilledby
0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,...,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship
1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,...,1,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship
2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,...,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN
3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,...,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship
4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,...,1,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128970,406-6001380-7673107,05-31-22,Shipped,Amazon,Amazon.in,Expedited,JNE3697,JNE3697-KR-XL,kurta,XL,...,1,INR,517.00,HYDERABAD,TELANGANA,500013.0,IN,NaN,False,NaN
128971,402-9551604-7544318,05-31-22,Shipped,Amazon,Amazon.in,Expedited,SET401,SET401-KR-NP-M,Set,M,...,1,INR,999.00,GURUGRAM,HARYANA,122004.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN
128972,407-9547469-3152358,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0157,J0157-DR-XXL,Western Dress,XXL,...,1,INR,690.00,HYDERABAD,TELANGANA,500049.0,IN,NaN,False,NaN
128973,402-6184140-0545956,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0012,J0012-SKD-XS,Set,XS,...,1,INR,1199.00,Halol,Gujarat,389350.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN


In [30]:
stylestock = pd.read_csv("../db/stylestock.csv")
orders = pd.merge(left = orders, right = stylestock, on = "Style", how = "inner").drop(columns=["Unnamed: 0"])
orders["Sales"] = orders.Qty * orders.Amount 

In [5]:
orders.ASIN.value_counts()

ASIN
B09SDXFFQ1    773
B09SDY8DCT    657
B09SDYQ3WG    587
B081WSCKPQ    535
B08XNJG8B1    507
             ... 
B089G1RT1M      1
B099S66DC2      1
B099S84F9Z      1
B09KXRCJ16      1
B08394R5YR      1
Name: count, Length: 5836, dtype: int64

In [6]:
popular_styles = orders.Style.value_counts().reset_index()[
    orders.Style.value_counts().reset_index()["count"] > 100
].Style.values.tolist()

In [7]:
orders.loc[~orders.Style.isin(popular_styles), "Style"] = "Other"

In [36]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots 

product_list = orders.Style.unique()

fig = make_subplots(rows=3, cols=2)

for i in range(3):
    for j in range(2): 
        item = np.random.choice(product_list) 
        product = orders[orders.Style == item] 
        product = product.groupby("Date")["Amount"].mean().reset_index()
        trace = go.Scatter(x=product["Date"], y=product["Amount"], mode="lines", name=item)
        fig.add_trace(trace, row=i+1, col=j+1)

fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.show()


In [31]:
orders["Date"] = pd.to_datetime(orders.Date) 
sales = orders.groupby("Date").sum()["Sales"].reset_index()
px.line(sales, x = 'Date', y = 'Sales')

In [27]:
orders["Date"] = pd.to_datetime(orders.Date) 
qty = orders.groupby("Date").sum()["Qty"].reset_index()
px.line(qty, x = 'Date', y = 'Qty')

In [33]:
orders["Cat-Size"] = orders.apply(lambda ds: f"{ds.Category}-{ds.Size}", axis=1)

In [45]:
J0117 = orders[orders.Style == 'J0117']
J0117["Amount"].unique()

array([518.  , 493.33,    nan,   0.  , 499.  , 475.24, 545.  , 519.05,
       522.  , 512.  , 487.  , 463.81, 726.  , 691.43])